## Preprocessing

In [63]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [64]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=["EIN", "NAME"], inplace=True)


In [65]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()

# Display unique values
unique_values


APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [66]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# Display value counts
application_type_counts


APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [67]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# Set the cutoff value
cutoff_value = 500

# Determine which application types to replace
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()




APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [68]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Display value counts
classification_counts


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [69]:
# You may find it helpful to look at CLASSIFICATION value counts >1
# Filter classification_counts for counts > 1
classification_counts_gt1 = classification_counts[classification_counts > 1]

# Display value counts greater than 1
classification_counts_gt1


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [70]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# Set the cutoff value
cutoff_value = 1000

# Determine which classifications to replace
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()





CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [71]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_encoded = pd.get_dummies(application_df)



In [72]:
# Split our preprocessed data into our features and target arrays
# Features are all columns except the target column
X = application_df_encoded.drop(columns=["IS_SUCCESSFUL"])

# Target is the "IS_SUCCESSFUL" column
y = application_df_encoded["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)



In [73]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)



## Compile, Train and Evaluate the Model

In [74]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer with 128 units
nn.add(tf.keras.layers.Dense(units=128, activation="relu", input_dim=X_train.shape[1]))

# Second hidden layer with 64 units
nn.add(tf.keras.layers.Dense(units=64, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()





c:\Users\bukol\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 128)            │         5,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,953 (54.50 KB)

 Trainable params: 13,953 (54.50 KB)

 Non-trainable params: 0 (0.00 B)

In [75]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [86]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, shuffle=True, verbose=2)
fit_model

Epoch 1/100
804/804 - 2s - 3ms/step - accuracy: 0.7428 - loss: 0.5310
Epoch 2/100
804/804 - 2s - 3ms/step - accuracy: 0.7431 - loss: 0.5303
Epoch 3/100
804/804 - 2s - 3ms/step - accuracy: 0.7437 - loss: 0.5302
Epoch 4/100
804/804 - 3s - 3ms/step - accuracy: 0.7439 - loss: 0.5304
Epoch 5/100
804/804 - 2s - 3ms/step - accuracy: 0.7425 - loss: 0.5297
Epoch 6/100
804/804 - 3s - 3ms/step - accuracy: 0.7439 - loss: 0.5294
Epoch 7/100
804/804 - 2s - 3ms/step - accuracy: 0.7435 - loss: 0.5299
Epoch 8/100
804/804 - 2s - 3ms/step - accuracy: 0.7433 - loss: 0.5301
Epoch 9/100
804/804 - 3s - 3ms/step - accuracy: 0.7423 - loss: 0.5305
Epoch 10/100
804/804 - 3s - 3ms/step - accuracy: 0.7427 - loss: 0.5300
Epoch 11/100
804/804 - 3s - 3ms/step - accuracy: 0.7435 - loss: 0.5293
Epoch 12/100
804/804 - 2s - 3ms/step - accuracy: 0.7434 - loss: 0.5306
Epoch 13/100
804/804 - 2s - 3ms/step - accuracy: 0.7437 - loss: 0.5297
Epoch 14/100
804/804 - 2s - 3ms/step - accuracy: 0.7437 - loss: 0.5304
Epoch 15/100
80

In [87]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Print the model's loss and accuracy
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Print the model's training history
print(fit_model.history)

268/268 - 1s - 3ms/step - accuracy: 0.7258 - loss: 0.6816
Loss: 0.6816497445106506, Accuracy: 0.7258309125900269
{'accuracy': [0.7428471446037292, 0.7431192398071289, 0.7437412738800049, 0.7438578605651855, 0.7424584031105042, 0.7438578605651855, 0.7435469031333923, 0.7432747483253479, 0.7423028945922852, 0.7427304983139038, 0.7434691190719604, 0.7434302568435669, 0.7437023520469666, 0.7437023520469666, 0.7433136105537415, 0.7427304983139038, 0.7436246275901794, 0.7428471446037292, 0.7434691190719604, 0.7435857653617859, 0.7432358860969543, 0.7433525323867798, 0.7436246275901794, 0.7429248690605164, 0.7435469031333923, 0.7434691190719604, 0.7432358860969543, 0.743507981300354, 0.7433525323867798, 0.743507981300354, 0.743818998336792, 0.7434302568435669, 0.7438967227935791, 0.7440133690834045, 0.743663489818573, 0.7431192398071289, 0.743818998336792, 0.7432358860969543, 0.7440133690834045, 0.7434302568435669, 0.7436246275901794, 0.743663489818573, 0.7436246275901794, 0.743507981300354, 

In [89]:
# Export our model to HDF5 file
# Define the file path for saving the model
model_file = "AlphabetSoupCharity.h5"

# Save the model to HDF5 file
nn.save(model_file)

# Print a confirmation message
print(f"Model saved to {model_file}")

Model saved to AlphabetSoupCharity.h5
